In [1]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -a "Kenneth Brezinski" -v

Author: Kenneth Brezinski

Python implementation: CPython
Python version       : 3.9.10
IPython version      : 8.0.1



In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%watermark --iversions

numpy     : 1.22.2
matplotlib: 3.5.1
pandas    : 1.4.1



In [10]:
df = pd.read_csv('data/mobility.csv')
df.head()

,date,location_key,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential
0,2020-02-15,AE,0.0,4.0,5.0,0.0,2.0,1.0
1,2020-02-16,AE,1.0,4.0,4.0,1.0,2.0,1.0
2,2020-02-17,AE,-1.0,1.0,5.0,1.0,2.0,1.0
3,2020-02-18,AE,-2.0,1.0,5.0,0.0,2.0,1.0
4,2020-02-19,AE,-2.0,0.0,4.0,-1.0,2.0,1.0


In [13]:
is_CA = df['location_key'].apply(lambda x: x.startswith('CA') if isinstance(x, str) else False)
df_CA = df[is_CA]
print(df_CA.columns)

Index(['date', 'location_key', 'mobility_retail_and_recreation',
       'mobility_grocery_and_pharmacy', 'mobility_parks',
       'mobility_transit_stations', 'mobility_workplaces',
       'mobility_residential'],
      dtype='object')


In [14]:
## only categroized by provinces, not subregions

df_CA['location_key'].unique()

array(['CA', 'CA_AB', 'CA_BC', 'CA_MB', 'CA_NB', 'CA_NL', 'CA_NS',
       'CA_NT', 'CA_NU', 'CA_ON', 'CA_PE', 'CA_QC', 'CA_SK', 'CA_YT'],
      dtype=object)

In [ ]:
tmp.columns

In [ ]:
# sanity check to check which columns have missing elements

is_CA = df['location_key'].apply(lambda x: x.startswith('CA') if isinstance(x, str) else False)
df_CA = df[is_CA]

# loop through provinces
for province in df_CA['location_key'].unique():
    
    # fetch provincial data
    tmp = df_CA[df_CA['location_key'] == province]
    
    # loop through mobility metrics
    for metric in tmp.columns:
        # check if column has missing data
        if tmp[metric].isnull().sum() == 0:   
            continue
        # print missing occurances
        print(f"{province}:{metric}: {tmp[metric].isnull().sum()} missing")   

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

## performs quite well; better to just stick to StandardScalar
X = df_CA[df_CA['location_key'] == 'CA_ON'].iloc[:, 2:]

pipe = Pipeline([('scaler', StandardScaler()),
                 ('knn', KNNImputer(n_neighbors=5, weights="distance"))])
trans = pipe.fit_transform(X)

plt.plot(trans)